## **Questão 1**

#### **Explicando alguns termos**


##### **1) Backtest**

Backtest é o processo de testar uma estrátegia de investimentos ou modelo de trading usando dados históricos. O objetivo de um backtest é
avaliar como a estratégia elaborada teria se comportado no passado para, a partir disso, inferir como pode ser o potencial futuro dessa
estratégia.

##### **2) Dividendos**

Dividendos são pagamentos distribuidos por uma certa empresa a seus acionistas como uma parcela do lucro da empresa. Eles são uma forma de retorno de investimento para os acionistas e podem ser pagos em dinheiro ou em ações adicionais.

##### **3) Split**

Um split, ou split de ações, é um evento corporativo onde uma certa empresa decide dividir suas ações existentes em um número maior de ações. Isso geralmente é feito para tornar o preço dessas ações mais acessível aos investidores, sem alterar o valor total da empresa.

##### **4) Reverse split**

Um reverse split é um evento corporativo em que uma empresa reduz o número de suas ações em circulação ao consolidar várias ações existentes em uma única ação nova. Isso acaba resultando em um aumento no preço das ações de tal empresa, sem alterar o valor total de mercado da mesma.

##### **5) Spin-off**

Um spin-off é um evento corporativo em que uma empresa cria uma nova empresa independente, separando uma parte de seus negócios. Em tal evento, os acionistas da empresa original geralmente recebem ações da nova empresa, proporcionais a sua participação na empresa original.

##### **6) Adjustment close**

O adjustment close é basicamente o preço de uma ação ajustado a ocorrência de eventos corporativos como dividendos, splits, reverse splits e spin-offs. Esse tipo de ajuste é feito para fornecer uma visão mais precisa do desempenho de uma ação ao longo do tempo, levando em conta as mudanças que afetam o preço.

#### **Resolução da questão**

##### **Instalação de bibliotecas**

In [ ]:
# Instala a bibliotecas que ser utilizada para a importação dos dados necessários para a resolução das questões abaixo.
%pip install yfinance

##### **Importação de bibliotecas**

In [ ]:
# Importa as bibliotecas que serão utilizadas na resolução das questões abaixo.
import yfinance as yf
# Importa a biblioteca que sera utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que sera utilizada para a manipulação de datas.
from datetime import datetime
# Importa as bibliotecas que serão utilizadas para a criação de gráficos.
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns


##### **Setup**

In [ ]:
# Cria uma variável de configuração que conterá dados úteis na resolução das questões abaixo.
setup = {
    "start_date": datetime(2018,1,1).date(),
    "end_date": datetime(2022,12,31).date(),
    "tickers": {
        "Vale":"VALE3.SA",
        "Banco do Brasil": "BBAS3.SA",
        "Bradesco":"BBDC3.SA",
        "Itau":"ITUB4.SA",
        "BTG Pactual":"BPAC11.SA",
        "Banco Pan":"BPAN4.SA",
        "Banco de Brasilia":"BSLI3.SA",
        "Banco BMG":"BMGB4.SA",
        "Santander":"SANB11.SA",
        "BR Partners":"BRBI11.SA"
    }
}

##### **Questão 1 - a)** 

*Obtenha os dados diários das ações da Vale (VALE3)*

In [ ]:
def get_ticker_data(ticker, start_date, end_date):
    '''
        Description: Essa função é responsável por obter alguns dados sobre as ações de uma determinada empresa em um certo período.
        
        Args:
            ticker (string): Identificador da empresa que se deseja obter os dados.
            start_date (string): Primeira data em que se deseja obter os dados sobre as ações da empresa.
            end_date (string): última data em que se deseja obter os dados sobre as ações da empesa.
        
        Return:
            data (pandas.DataFrame): DataFrame que contém alguns dados sobre as ações da empresa em questão.
    '''
    try:
        # Obtem o DataFrame que contém alguns dados sobre as ações da empresa solicitada.
        data = yf.download(ticker, start_date, end_date)
    except Exception as e:
        print(f"Erro ao baixar os dados do ticker {ticker}: {e}")
    
    # Retorna o DataFrame que contém os dados obtidos.
    return data

In [ ]:
# Salva em uma variável o DataFrame que contém os dados obtidos.
VALE_data = get_ticker_data(setup["tickers"]["Vale"], setup["start_date"], setup["end_date"])

In [ ]:
# Exibo os dados obtidos.
VALE_data

##### **Questão 1 - b)**
*Obtenha dados diários de um conjunto de 10 ações da B3 (inclua a VALE3 e escolha as outras 9); agrupe-as da forma que preferir. Depois, filtre para obter um dataframe apenas com o preço de fechamento ajustado*

In [ ]:
def get_tickers_data(tickers_list, start_date, end_date):
    '''
        Description: ...
        
        Args:
            tickers_list (list): Lista contendo as strings que representam os tickers das empresas cujos dados das ações serão obtidos.
            start_date (datetime.date()): Data inicial do intervalo onde os dados serão extraidos.
            end_date (datetime.date()): Data final do intervalo onde os dados serão extraidos.
        
        Return:
            result (pd.DataFrame): DataFrame contendo os preços de fechamento ajustado durante o período compreendido entre "start_date"
            e "end_date" de todas as empresas cujo ticker esta em "tickers_list".       
            
        Obs: Pode ocorrer que alguma das empresas cujo ticker esta em "tickers_list" tenha sido listada na B3 após a data "start_date", caso isso
        ocorra, o preço de fechamento ajustado de tal empresa será setado como 0 em todas as datas a partir de "start_date" até a data em que 
        essa empresa foi listada na B3.
    '''
    
    # Por conta da variável "index_union" que será criada mais abaixo, trataremos o download dos dados do primeiro ticker de forma separada.
    # Obtem os dados do primeiro ticker.
    first_ticket = yf.download(tickers_list[0], start_date, end_date)
    # Cria uma variável que conterá a união (no sentido de conjuntos) de todos os índices de cada um dos tickers pertencentes a "ticker_list".
    # Ou seja, essa variável guardará os índices (que são as datas onde ocorreu negociação na B3) do ticker pertencente a "ticker_list" que foi
    # listado primeiro na B3. Faremos isso visando a padronização citada na observação da docstring dessa função.
    index_union = first_ticket.index
    # Cria uma lista para guardar as séries que representam o "Adj Close" de cada um dos tickers em "ticker_list".
    series_list = []
    # Adiciona a série "Adj Close" do primeiro ticker em "ticker_list" a lista "series_list".
    series_list.append(first_ticket['Adj Close'])
    
    # Percorre todos os tickers em ticker_list com exceção do primeiro.
    for ticker in tickers_list[1:]:
        try:
            # Faz o download dos dados ticker em questão no período compreendido entre "start_date" e "end_date".
            ticker_data = yf.download(ticker, start_date, end_date)
        except Exception as e:
            print(f"Erro ao baixar os dados do ticker {ticker}: {e}")
            continue
        # Faz a união dos índices do ticker em questão com todos os índices dos tickers que foram unidos até então.
        index_union = index_union.union(ticker_data.index)
        # Adiciona a série "Adj Close" do ticker em questão a lista "series_list".
        series_list.append(ticker_data['Adj Close'])
        
    # Reindexa todas as séries contidas em "series_list" levando em conta os índices em "index_union" e preenche eventuais valores faltantes
    # como zero.
    reindexed_serie_list = [serie.reindex(index_union, fill_value = 0) for serie in series_list]
    
    # Concatena todas as séries que foram reindexadas acima em um único DataFrame e usa o ticker de cada uma das séries como chave para
    # criar níveis superiores.
    result = pd.concat(reindexed_serie_list, axis=1, keys=tickers_list)
    
    # Retorna o DataFrame criado acima.
    return result

In [ ]:
# Cria uma lista que contém todos os tickers que estão na variável de configuração "setup".
tickers_list = list(setup["tickers"].values())
# Obtem alguns dados relacionados as ações de cada um dos tickers presentes na "tickers_list" criada acima.
tickers_data = get_tickers_data(tickers_list, setup['start_date'], setup['end_date'])

In [ ]:
# Exibe os dados obtidos.
tickers_data

##### **Questão 1 - c)**

*Faça um único gráfico contendo a série temporal do preço de fechamento ajustado de cada uma das 10 ações que você escolheu no item anterior.*

In [ ]:
def plot_all_adjusted_close_prices(data):
    '''
        Description:
            Esta função plota ao longo do tempo os preços de fechamento ajustados das ações presentes no DataFrame "data".
        Args:
            data (pandas.DataFrame): DataFrame contendo os preços de fechamento ajustado das ações. Tal DataFrame deve ter datas como índices e
            deve ter cada uma de suas colunas representando os preços de fechamento ajustados de um certo ticket.
        Return:
            None: A função exibe o gráfico gerado, mas não retorna nenhum valor.
    '''    
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data.index.tolist()
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for ticker in data.columns:
        # Cria uma lista com os preços de fechamento ajustados das ações do ticker em questão.
        y_axis = data[ticker].values
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=y_axis, mode="lines", name=ticker))
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        title='Série temporal do preço de fechamento ajustado de algumas ações',
        xaxis_title='Data',
        yaxis_title='Valor da Ação',
        legend_title='Ações',
        hovermode='x unified',
        clickmode='event+select'
    )

    # Callback para destacar a linha ao clicar na legenda.
    fig.update_traces(
        marker=dict(size=12),
        selector=dict(mode='markers')
    )

    # Exibe o gráfico criado.
    fig.show()

In [ ]:
# Plota os gráficos (Data x Valor da Ação) de cada uma das empresas cujo ticker está presente em "tickers_data".
plot_all_adjusted_close_prices(tickers_data)

##### **Questão 1 - d)**

*Escolha uma das 10 ações e faça um gráfico contendo a média móvel dos últimos 20 dias do preço de fechamento, mais as bandas de Bollinger (dois desvios para cima e dois desvios para baixo). O que a média móvel representa e porque usar as bandas de Bollinger?*

**O que a média móvel representa e porque usar as bandas de Bollinger?**

*Resposta aqui*

##### **Questão 1 - e)**
*Faça um único gráfico contendo o preço de fechamento da ação escolhida e seu Volume (negociado no dia)*

##### **Questão 1 - f)**

*Faça um gráfico de dispersão (do preço de fechamento ajustado) entre o ativo escolhido anteriormente e outro de sua escolha. Qual sua interpretação?*

**Interpretação do gráfico de dispersão acima**

*Resposta aqui*

##### **Desafio: Faça um gráfico candle stick da ação escolhida.**